In [14]:
#[Problem 1] Sharing and executing the official tutorial mode
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])

# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8694 - loss: 0.4559 - val_accuracy: 0.9632 - val_loss: 0.1288
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9628 - loss: 0.1272 - val_accuracy: 0.9727 - val_loss: 0.0931
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.9756 - loss: 0.0848 - val_accuracy: 0.9722 - val_loss: 0.0890
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.9829 - loss: 0.0603 - val_accuracy: 0.9767 - val_loss: 0.0769
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9859 - loss: 0.0473 - val_accuracy: 0.9777 - val_loss: 0.0768


In [13]:
#[Problem 2] (Advance assignment) Execute various methods
from PIL import Image
import matplotlib.pyplot as plt

detect_fn = tf.saved_model.load("path/to/saved_model")

img = Image.open("image.jpg").convert("RGB")
img_np = np.array(img)
input_tensor = tf.convert_to_tensor(img_np)[tf.newaxis, ...]

outputs = detect_fn(input_tensor)

boxes = outputs["detection_boxes"][0].numpy()
classes = outputs["detection_classes"][0].numpy().astype(np.int32)
scores = outputs["detection_scores"][0].numpy()

keep = scores > 0.5
boxes, classes, scores = boxes[keep], classes[keep], scores[keep]
print("Detections:", list(zip(classes.tolist(), scores.tolist())))

def draw_box(ax, box):
    h, w = img_np.shape[:2]
    y1, x1, y2, x2 = box
    ax.add_patch(plt.Rectangle((x1*w, y1*h), (x2-x1)*w, (y2-y1)*h,
                               fill=False, linewidth=2))

fig = plt.figure(figsize=(8,6)); ax = plt.gca(); ax.imshow(img_np); ax.axis('off')
for b in boxes: draw_box(ax, b)
plt.show()


OSError: SavedModel file does not exist at: path/to/saved_model/{saved_model.pbtxt|saved_model.pb}

In [15]:
#[Problem 3] Learning Iris (binary classification) with Keras


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

mask = y != 0
X = X[mask]
y = y[mask]

y = LabelEncoder().fit_transform(y)  # 0 or 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(8, activation="relu", input_shape=(4,)),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=50, verbose=0)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Binary Iris accuracy: {acc:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Binary Iris accuracy: 0.8500


In [17]:
#[Problem 4] Learn Iris (multi-level classification) with Keras

iris = load_iris()
X = iris.data
y = to_categorical(iris.target, 3)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(10, activation="relu", input_shape=(4,)),
    Dense(3, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=50, verbose=0)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Multi-class Iris accuracy: {acc:.4f}")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Multi-class Iris accuracy: 0.6333


In [18]:
#[Problem 5] Learning House Prices with Keras
import pandas as pd


data = pd.read_csv("train.csv")
X = data.drop("SalePrice", axis=1).select_dtypes(include=[np.number]).fillna(0)
y = data["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.fit(X_train, y_train, epochs=50, verbose=0, validation_split=0.1)

loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"House Prices MAE: {mae:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


House Prices MAE: 96442.22


In [8]:
#[Problem 6] Learning MNIST with Keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"MNIST accuracy: {acc:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 29s 33ms/step - accuracy: 0.8973 - loss: 0.3577 - val_accuracy: 0.9798 - val_loss: 0.0724
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 32s 38ms/step - accuracy: 0.9806 - loss: 0.0644 - val_accuracy: 0.9845 - val_loss: 0.0548
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - accuracy: 0.9884 - loss: 0.0386 - val_accuracy: 0.9865 - val_loss: 0.0535
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.9918 - loss: 0.0262 - val_accuracy: 0.9877 - val_loss: 0.0468
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.9953 - loss: 0.0167 - val_accuracy: 0.9863 - val_loss: 0.0548
MNIST accuracy: 0.9818


In [19]:
#[Problem 7] (Advance assignment) Rewriting to PyTorch

import torch
import torch.nn as nn
import torch.optim as optim


iris = load_iris()
X, y = iris.data, iris.target
mask = y != 0
X, y = X[mask], y[mask]
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train, X_test = scaler.fit_transform(X_train), scaler.transform(X_test)

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

class IrisBinaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

model = IrisBinaryNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

with torch.no_grad():
    y_pred_test = model(X_test).round()
    acc = (y_pred_test.eq(y_test).sum().item()) / len(y_test)
print(f"PyTorch Binary Iris accuracy: {acc:.4f}")


PyTorch Binary Iris accuracy: 0.8500



#[Problem 8] (Advance assignment) Comparison of frameworks

TensorFlow is a very powerful and mature deep learning library that offers strong visualization capabilities and multiple options for high-level model development. It provides production-ready deployment solutions and supports mobile platforms, making it an excellent choice for developers who want to build models for real-world applications. TensorFlow is particularly useful when models need to be deployed on mobile devices, when robust community support and comprehensive documentation are desired, when rich learning resources such as MOOCs are needed, when TensorBoard visualization is important, or when large-scale distributed model building is required.

PyTorch, on the other hand, is a fully featured framework for building deep learning models and is commonly used in applications such as image recognition and natural language processing. Written in Python, PyTorch is relatively easy for most machine learning developers to learn and use. It is distinctive for its excellent GPU support and its use of reverse-mode automatic differentiation, which allows computation graphs to be modified dynamically during runtime. This flexibility makes PyTorch particularly popular for fast experimentation and rapid prototyping.

PyTorch was developed by Facebook AI Research along with several other labs. The framework combines efficient GPU-accelerated backend libraries from Torch with an intuitive Python frontend that emphasizes rapid prototyping, readable code, and support for a wide variety of deep learning models. Developers can use familiar imperative programming while still taking advantage of computation graphs. Since its open-source release in 2017, PyTorch’s Pythonic approach has made it a favorite among machine learning researchers and developers alike.

Although still relatively young compared to TensorFlow, PyTorch has grown rapidly and is especially suitable for research or projects where strict production requirements are not critical. It provides an excellent development and debugging experience and appeals to developers who prefer the flexibility and expressiveness of Python.